### Create Monthly Time Series DataFrames (and pickle.dump them)

In [1]:
import os
import warnings
import random

import numpy as np
import pandas as pd
import iris
from iris import cube
from iris import pandas
import cftime

from Functions import *
from Global_Variables import *

In [2]:
obs_dict = {}
for obsname in [f.name for f in os.scandir(OBSERVATIONS_PATH) if f.is_dir()]:
    for dirpath, subdirs, files in os.walk(os.path.join(OBSERVATIONS_PATH, obsname)):
        for filename in files:
            if 'Burned_Percentage.nc' in filename:
                obs_dict[obsname] = os.path.join(dirpath, filename)

In [3]:
model_dict = {}
for modelname in [f.name for f in os.scandir(MODEL_PATH) if f.is_dir()]:
    model_dict[modelname] = {}
    for dirpath, subdirs, files in os.walk(os.path.join(MODEL_PATH, modelname)):
        for filename in files:
            if filename.endswith('gswp3-w5e5_obsclim_histsoc_default_burntarea-total_global_monthly_1901_2019.nc'):
                model_dict[modelname]['obsclim'] = os.path.join(dirpath, filename)
            elif filename.endswith('gswp3-w5e5_counterclim_histsoc_default_burntarea-total_global_monthly_1901_2019.nc') \
            or filename.endswith('gswp3-w5e5_counterclim_histsoc_1901co2_burntarea-total_global_monthly_1901_2019.nc'): # This one should not be there, files should be named via the convention above
                model_dict[modelname]['counterclim'] = os.path.join(dirpath, filename)

The following two cells take quite some time to execute. Their results are stored in pandas DataFrames and pickle dumped under Results. They are used in in the following notebooks.

Observations

In [4]:
warnings.filterwarnings('ignore') # Surpress some unimportant warnings
obs_df_list = []

for obs_name, obs_loc in obs_dict.items():
    obs = iris.load_cube(obs_loc)
    time = obs.coords("time")[0]
    start_date = cftime.num2date(time.points, time.units.name, calendar=time.units.calendar)[0]
    dates = np.array(list(map(lambda x: np.datetime64(f'{start_date.year}-{start_date.month:02d}') + np.timedelta64(int(x),'M'), np.arange(len(time.points)))))
    index = pd.DatetimeIndex(data=dates, name='Date')
    columns = AR6_masks.keys()
    obs_df = pd.DataFrame(data=np.full((index.size, len(columns)), np.nan), index=index, columns=columns)

    for idx, (regionname, region_mask) in enumerate(AR6_masks.items()):
        obs_df.loc[slice(None), regionname] = to_timeseries(mask_region(obs, region_mask)).data
    
    obs_df_list.append(obs_df)
    
names = ['Observation', 'Region']  
AR6_indices = {name: idx for idx, name in enumerate(AR6_masks.keys())}
obs_df = pd.concat(obs_df_list, axis=1, keys=obs_dict.keys(), names=names).reorder_levels(names[::-1], axis=1).sort_values(by='Region', axis=1, key=lambda col: col.apply(lambda x: AR6_indices[x]))
obs_df.to_pickle(os.path.join(RESULTS_PATH, 'AR6_obs_df.pkl')) 
warnings.filterwarnings('default')

Simulations

In [5]:
warnings.filterwarnings('ignore') # Surpress some unimportant warnings

random_model_name = random.choice(list(model_dict.keys()))

dates = np.array([np.datetime64('1901-01') + np.timedelta64(int(idx),'M') for idx in range(preprocess_model(iris.load_cube(model_dict[random_model_name]['obsclim']), random_model_name).coords("time")[0].points.size)])
index = pd.DatetimeIndex(data=dates, name='Date')
model_tuples = [(regionname, modelname) for regionname in AR6_masks.keys() for modelname in model_dict.keys()]
multi_index = pd.MultiIndex.from_tuples(model_tuples, names=['Region', 'Model'])

obsclim_df = pd.DataFrame(data=np.full((index.size, multi_index.size), np.nan), index=index, columns=multi_index)
counterclim_df = pd.DataFrame(data=np.full((index.size, multi_index.size), np.nan), index=index, columns=multi_index)

for model_name, model_dict_path in model_dict.items():
    obsclim = preprocess_model(iris.load_cube(model_dict_path['obsclim']), model_name)
    counterclim = preprocess_model(iris.load_cube(model_dict_path['counterclim']), model_name)
    
    for idx, (regionname, region_mask) in enumerate(AR6_masks.items()):
        for sim, dfs in ((obsclim, obsclim_df), (counterclim, counterclim_df)):
            model_region = to_timeseries(mask_region(sim, region_mask))            
            dfs.loc[:, (regionname, model_name)] = iris.pandas.as_series(model_region).values

obsclim_df.to_pickle(os.path.join(RESULTS_PATH, 'AR6_obsclim_df.pkl'))
counterclim_df.to_pickle(os.path.join(RESULTS_PATH, 'AR6_counterclim_df.pkl'))

warnings.filterwarnings('default')